# Assignment 2

In this assignment you will implement ResNet18.
Read the comments carefully and insert your code where you see: <br><br><b>##### START OF YOUR CODE #####</b><br><br><b>##### END OF YOUR CODE #####</b><br><br>or for the inline codes you will see<br><br><b>##### INSERT YOUR CODE HERE #####</b>

### The architecture of ResNet-18 is shown in the table.
First, we will define a convolutional block with skip connection. Then, create the model using these blocks.<br><br>
<img src="https://www.researchgate.net/profile/Paolo-Napoletano/publication/322476121/figure/tbl1/AS:668726449946625@1536448218498/ResNet-18-Architecture.png" width="500" alt="ResNet18 Architecture">

<br><sup>Image ref: Napoletano, Paolo, et al. ‘Anomaly Detection in Nanofibrous Materials by CNN-Based Self-Similarity’. Sensors (Basel, Switzerland), vol. 18, 01 2018, https://doi.org10.3390/s18010209.</sup>

#### I. ConvBlock
<img src="https://www.researchgate.net/publication/334301817/figure/fig3/AS:778452965801986@1562609058538/Residual-block-of-ResNet18-with-a-1-1-convolutional-mapping-based-residual-unit-and.png"><br>
ResNet consists of convolutional (a) and identity (b) blocks. For ResNet-18 we will only use convolutional blocks. In this step you will write a class for convolutional block. The arguments will be:

* ch_in: input channels
* ch_out: output channels
* s: strides
* act: activation function

The options for activation function are "relu", "leaky_relu" and "gelu".
<br><br>
<sup>Image ref: Owais, Muhammad, et al. ‘Artificial Intelligence-Based Classification of Multiple Gastrointestinal Diseases Using Endoscopy Videos for Clinical Diagnosis’. Journal of Clinical Medicine, vol. 8, 07 2019, p. 986, https://doi.org10.3390/jcm8070986.</sup>

In [1]:
## 2022400354
# Yiğit SARIOĞLU
import torch
from torch import nn
from torch.nn import functional as F

class ConvBlock(nn.Module):
    def __init__(self, ch_in, ch_out, s, act):
      super(ConvBlock,self).__init__()

      ##### START OF YOUR CODE #####

       # Initialize layers

       # this is the first convolutional layer of the block.
       #it performs a 2D convolution operation with a specified number of input channel, output channel,kernel size ,stride and padding
      self.conv1 = nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=s, padding=1)
      self.bn1 = nn.BatchNorm2d(ch_out)

      # this is the second convolutional layer of the block.
      # it has the same number of input and output channels (ch_out) and uses a kernel size of 3x3, a stride of 1, and padding of 1.
      self.conv2 = nn.Conv2d(ch_out, ch_out, kernel_size=3, stride=1, padding=1)
      self.bn2 = nn.BatchNorm2d(ch_out)

      if act == "relu":
        self.activation = nn.ReLU()  #if act is relu it uses the rectified linear unit(ReLU) activation function.
      elif act == "leaky_relu":
        self.activation = nn.LeakyReLU()  #if act is leaky relu it uses the  Leaky ReLU activation function..
      elif act == "gelu":
        self.activation = nn.GELU() # if act is gelu it uses the GELU (Gaussian Error Linear Unit) activation function.
      else:
        raise ValueError("Invalid activation fuction...") # if none of them, it gives an error



      ##### END OF YOUR CODE #####

    def forward(self, X):
      ##### START OF YOUR CODE #####

      # First conv layer
      out = self.conv1(X)
      out = self.bn1(out)
      out = self.activation(out)

      # Second convolutional layer
      out = self.conv2(out)
      out = self.bn2(out)

      ##### END OF YOUR CODE #####
      return X

#### II. ResNet18 class
Use the ConvBlock class to create ResNet18.
* Add batch normalization and activation function after all convolutional layers.
* Examine the output sizes in the table and use paddings and strides where needed. (Note that the input size is 224 x 224 in this example)
* Add a drop-out layer after average pooling.
* Fully connected layer should be 512 x 1 as we have only 2 classes and we will use sigmoid function as the final activation layer.

<img src="https://www.researchgate.net/profile/Paolo-Napoletano/publication/322476121/figure/tbl1/AS:668726449946625@1536448218498/ResNet-18-Architecture.png" width="500" alt="ResNet18 Architecture">

In [2]:
class ResNet18(nn.Module):
    def __init__(self, act, drop_rate):
      super(ResNet18, self).__init__()
      # Initialize layers
      ##### START OF YOUR CODE #####

      # Initialize layers
      self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3) # this is the initial convolutional layer that takes a 224x224 input image with 3 input channels
      self.bn1 = nn.BatchNorm2d(64)
      self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

      # these represents the convolutional blocks in the ResNet architecture.
      self.conv2_x = self._make_layer(64, 64, 2, act)  # self.conv2_x has two residual blocks with 64 input channels
      self.conv3_x = self._make_layer(64, 128, 2, act, s=2)
      self.conv4_x = self._make_layer(128, 256, 2, act, s=2)
      self.conv5_x = self._make_layer(256, 512, 2, act, s=2) # self.conv5_x has two residual blocks with 512 input channels

      self.avgpool = nn.AdaptiveAvgPool2d((1, 1)) # this layer performs adaptive average pooling
      self.fc = nn.Linear(512, 1)  # this is the fully connected layer at the end of the network, it maps the 512-dimensional feature vector to a single output unit
      self.dropout = nn.Dropout(drop_rate) # dropout layer with a specified dropout rate helps prevent overfitting

      ##### END OF YOUR CODE #####

    def forward(self, X):
      ##### START OF YOUR CODE #####
      X = self.conv1(X)
      X = self.bn1(X)
      X = F.relu(X)
      X = self.pool(X)

      X = self.conv2_x(X)
      X = self.conv3_x(X)
      X = self.conv4_x(X)
      X = self.conv5_x(X)

      X = self.avgpool(X)
      X = X.view(X.size(0), -1)

      X = self.fc(X)
      X = self.dropout(X)
      return X

    def _make_layer(self, in_channels, out_channels, num_blocks, act, s=1):

        layers = []
        layers.append(ConvBlock(in_channels, out_channels, s, act))
        for _ in range(1, num_blocks):
            layers.append(ConvBlock(out_channels, out_channels, 1, act))
        return nn.Sequential(*layers)

      ##### END OF YOUR CODE #####


In [3]:
# Print the model
model = ResNet18("relu", .5)
print(model)

ResNet18(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2_x): Sequential(
    (0): ConvBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
    )
    (1): ConvBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05,